# Import libraries

In [2]:
import pandas as pd
import numpy as np
import os

# Feature Correspondence

In [9]:
fcorr = pd.read_csv('../tables/featureCorrespondence_e5.csv')

fcorr.rename(columns={"Unnamed: 0":'Features'}, inplace=True)

features = fcorr[['Features']]

fcorr['Features'] = features

fcorr.set_index('Features', inplace=True)

print(fcorr.shape)

fcorr.head()

(532, 8)


,BNC,Bog_T0,III_Bog_N1,III_Bog_N2,II_Bog_N1,II_Bog_N2,I_Bog_N1,I_Bog_N2
Features,,,,,,,,
FT001,4.281247e+06,1.447325e+06,1.964509e+06,1.755049e+06,1.470808e+06,1.568535e+06,1.699758e+06,1.584472e+06
FT002,1.802815e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
FT003,1.245198e+06,4.096185e+04,1.152065e+05,7.553663e+04,4.407506e+04,6.536492e+04,3.881721e+04,8.149627e+04
FT004,5.079143e+06,5.244796e+05,1.647061e+06,9.352173e+05,1.169840e+06,1.296817e+06,1.170791e+06,1.407095e+06
FT005,1.100489e+07,3.833114e+03,2.443153e+03,4.821497e+03,6.159005e+03,5.007683e+03,2.887445e+03,0.000000e+00


# Only MS2

`featureCorrespondence.csv` all the features (== MS1), but all of them had an MS2.

Normally, I'll do this step after the MS2 annotation, but for this tutorial, we'll only work with the features that have MS2.

In [10]:
list_ms2 = open('../data/ms2spectra_consensus_e5.mgf', 'r').read().split('\n')
list_ms2 = [x.split('=')[-1] for x in list_ms2 if x.startswith('TITLE')]
print(len(list_ms2))
list_ms2[:5]

280


['FT001', 'FT004', 'FT008', 'FT011', 'FT013']

## Filter 

In [11]:
fcorr_ms2 = fcorr.copy()

fcorr_ms2 = fcorr_ms2.reset_index()

fcorr_ms2 = fcorr_ms2[fcorr_ms2['Features'].isin(list_ms2)]

fcorr_ms2 = fcorr_ms2.set_index('Features')

fcorr_ms2 = fcorr_ms2.replace(np.nan, 0)

## log transform
fcorr_ms2 = np.log10(fcorr_ms2+1)

print(fcorr_ms2.shape)

## filter rows that have variance == 0; will cause issues in the PCA
fcorr_ms2 = fcorr_ms2[fcorr_ms2.var(axis=1).astype(int) > 0.0]

print(fcorr_ms2.shape)

fcorr_ms2.to_csv('../tables/featureCorrespondence_MS2_e5.csv')

fcorr_ms2.head()

(280, 8)
(70, 8)


,BNC,Bog_T0,III_Bog_N1,III_Bog_N2,II_Bog_N1,II_Bog_N2,I_Bog_N1,I_Bog_N2
Features,,,,,,,,
FT050,7.452494,3.845745,6.349502,6.370515,6.757783,6.706847,6.846109,6.831937
FT056,6.756613,0.000000,6.157832,6.109161,6.102997,6.100061,6.004018,6.074969
FT068,6.639078,0.000000,3.450157,3.464679,0.000000,3.265679,4.268448,4.785439
FT099,7.580938,3.621763,6.341433,6.449305,6.540656,6.688573,6.657739,6.735765
FT106,7.526662,0.000000,0.000000,3.654829,0.000000,6.841935,0.000000,0.000000


In [7]:
fcorr_ms2.loc['FT0473', :]

BNC           7.695127
Bog_T0        3.838134
III_Bog_N1    6.275136
III_Bog_N2    6.370515
II_Bog_N1     6.761377
II_Bog_N2     6.706847
I_Bog_N1      6.846109
I_Bog_N2      6.831937
Name: FT0473, dtype: float64